API docs for the WikiSRATMicroService
===

**SRAT code is all in SQL, run from this notebook.**  
This code is within the GitHub repository. The result of this will be a table of NHDplus catchments. 

**Note on `ploadrate_total`**  
The microservice output says "tploadrate_total", however I [Mike] suspect this is a mistake. This is actually the local NHDplus catchment load. The stream concentrations take into account the watershed load, but this value is not returned. I went ahead and calculated this when I export the microservice results to PG for convenience / if we need the value later.

**Instances where load reduction is greater than the load created by an NHDplus catchment:**
- After weekly check in call, decided that it can be negative, so leave as raw value
- Otherwise can set threshold to where can't reduce loads beyond a proportion of the current load

In [1]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import json
import os
import psycopg2
from StringParser import StringParser
from DatabaseAdapter import DatabaseAdapter
from DatabaseFormatter import DatabaseFormatter
from DatabaseMakeTable import DatabaseMakeTable

# Run WikiSRATMicroService API

## Open Connection to WikiSRATMicroService Database

To fetch information to assist in formating requests to the WikiSRATMicroService API.

In [2]:
# GET THE DATABASE CONFIG INFORMATION USING A CONFIG FILE. THE FILE IS IN THE GITIGNORE SO WILL REQUIRE BEING SENT

config_file = json.load(open('db_config.json'))
PG_CONFIG = config_file['PG_CONFIG']

_host = PG_CONFIG['host'],
_database = PG_CONFIG['database'],
_user = PG_CONFIG['user'],
_password = PG_CONFIG['password'],
_port = PG_CONFIG['port']

In [3]:
# Create connection to database

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

## Fetch data for formating requests to the WikiSRATMicroService API


In [4]:
%%time

# GET THE MODELED LOADS FROM THE DRWI DATABASE, DERIVED FROM MMW MODEL RUNS

_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw order by huc12;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata))

484
CPU times: user 12.6 ms, sys: 5.73 ms, total: 18.4 ms
Wall time: 603 ms


In [5]:
_dbdata?

Type:        list
String form: [(1, '020401010101', Decimal('83.25'), Decimal('131587.70'), Decimal('144086.00'), Decimal('11916 <...> 00421087362510914, 0.0325741241610815, 0.124997522620959, 0.149069147756057, 0.0349152439841768)]
Length:      484
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


## Run WikiSRATMicroService API


In [6]:
# CREATE A COUPLE HELPER FUNCTIONS TO RUN THE MICROSERVICE
def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.args[0] if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
    }

def lambda_handler(event, context):
    try:
        data = StringParser.parse(event['body'])
        db = DatabaseAdapter(_database[0], _user[0], _host[0], _port, _password[0], _flag)
        input_array = DatabaseAdapter.python_to_array(data)
        return respond(None, db.run_model(input_array))
    except AttributeError as e:
        return respond(e)

In [7]:
%%time

# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION
# The database adapter routine flag can either be 'base' or 'restoration', depending on if you want these
# projects to be removed from the attenuation routine. Restoration projects come from what was enetered in
# through FieldDoc.

_flag = 'base'


# RUN THE HUC12s THROUGH THE MICROSERVICE
_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

_r = dict(lambda_handler({"body": _body},None))

CPU times: user 341 ms, sys: 45.6 ms, total: 387 ms
Wall time: 5.94 s


In [8]:
# Explore the API response
_r?

Type:        dict
String form: {'statusCode': '200', 'body': '{"huc12s": {"020401010101": {"huc12": "020401010101", "tpload_hp": <...> 15247}}}}}', 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}}
Length:      3
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [11]:
# Extract the NHD Loads from the response
_nhdloads = dict(json.loads(_r['body']))['huc12s']
_nhdloads?

Type:        dict
String form: {'020401010101': {'huc12': '020401010101', 'tpload_hp': 602.07665754054, 'tpload_crop': 360.72733 <...> 0436867554431356, 'tssloadrate_total': 25055.0654476434, 'tssloadrate_conc': 41.5994419315247}}}}
Length:      484
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [10]:
# Explore selection of data by HUC12
print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

{'4481881': {'comid': 4481881, 'tploadrate_total': 8.67678906666698, 'tploadate_conc': 0.00480867068316327, 'tnloadrate_total': 205.478173246534, 'tnloadrate_conc': 0.188308808608071, 'tssloadrate_total': 12430.9823731758, 'tssloadrate_conc': 10.6329585165398}, '4481681': {'comid': 4481681, 'tploadrate_total': 45.9269178229512, 'tploadate_conc': 0.0139356826495012, 'tnloadrate_total': 1098.12798966065, 'tnloadrate_conc': 0.333206840298741, 'tssloadrate_total': 128512.512354698, 'tssloadrate_conc': 38.9947698116636}, '4481279': {'comid': 4481279, 'tploadrate_total': 9.3369485692889, 'tploadate_conc': 0.0870941956538323, 'tnloadrate_total': 260.499630512306, 'tnloadrate_conc': 2.34191601922763, 'tssloadrate_total': 4135.72100063273, 'tssloadrate_conc': 49.6417992751781}, '4481935': {'comid': 4481935, 'tploadrate_total': 47.8207522003034, 'tploadate_conc': 0.107891339771179, 'tnloadrate_total': 1122.82652578667, 'tnloadrate_conc': 2.53327797292518, 'tssloadrate_total': 29233.9090668937, '

# Load Results into Database

Loading results into the database is a time consuming process (~20-30 minutes), so it only needs to be run once for every data update. 

### If you have done this for the latest data, skip this block.


In [9]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON

t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1
        
t

19496

## Save "Base" Model Results (i.e. no BMPs) to Database

In [5]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
# This uses an imported function to create the table. This is necessary to get the COMID geometries

# SET THE TABLE NAME AND CREATE TABLE
tablename_base = 'base_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_base)
new.make_table()


Table Created


In [6]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_base, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)
print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done


## Repeat for Restoration Results

Reruns the WikiSRATMicroService API with restoration BMPs and saves response to the database.

In [7]:
# NOW RUN THE ATTENUATION WITH THE BMPs ADDED IN
_flag = 'restoration'

_r = dict(lambda_handler({"body": _body},None))
print('done')

done


In [8]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON
_nhdloads = dict(json.loads(_r['body']))['huc12s']
t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1

In [9]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
tablename_rest = 'restoration_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_rest)
new.make_table()

Table Created


In [10]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_rest, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)

print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done


# ABOVE CODE ONLY NEEDS TO BE RUN ONCE

Use labs to get outline to collapse code to be run once. Further edits to the data frames for API output can be done here.

# Read Results in Pandas and Save to Local Storage

In [16]:
# PUT HERE ANY TABLE NAMES CREATED ABOVE, IN CASE THE KERNAL WAS RESTARTED AND YOU DON'T WANT TO RUN THE ABOVE AGAIN
tablename_rest = 'restoration_run'
tablename_base = 'base_run'

## Read from Database into Pandas with `psycopg2`
Custom function written by Mike, using the pyscopg2 library.

In [23]:
%%time

# LOAD THE DATABASE TABLES INTO PANDAS
# geom = NHDplus streamline
# catchment = NHDplus catchment

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    _cur = conn.cursor()
    try:
        _cur.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        _cur.close()
        return 1
    
    # We get a list of tupples
    tupples = _cur.fetchall()
    _cur.close()
    
    # Turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

colnames = ('comid','tploadrate_total','tploadate_conc','tnloadrate_total','tnloadate_conc','tssloadrate_total',
            'tssloadate_conc','catchment_hectares','watershed_hectares','tploadrate_total_ws','tnloadrate_total_ws',
            'tssloadrate_total_ws','maflowv','geom','geom_catchment', 'cluster', 'fa_name', 'sub_focusarea',
            'nord','nordstop', 'huc12')

cluster_names = ('Brandywine and Christina','Kirkwood - Cohansey Aquifer','Middle Schuylkill','New Jersey Highlands',
                 'Poconos and Kittatinny','Schuylkill Highlands','Upper Lehigh','Upstream Suburban Philadelphia')


base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = postgresql_to_dataframe(_PG_Connection, base_model_select, colnames)
rest_df = postgresql_to_dataframe(_PG_Connection, rest_model_select, colnames)

print('done')

done
CPU times: user 625 ms, sys: 901 ms, total: 1.53 s
Wall time: 30.4 s


In [28]:
# Note that most columns are read as string objects, and ...
# geometry datatypes are not compatbile with GeoPandas and common Python plotting functions.
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   comid                 19496 non-null  int64  
 1   tploadrate_total      19496 non-null  object 
 2   tploadate_conc        19496 non-null  object 
 3   tnloadrate_total      19496 non-null  object 
 4   tnloadate_conc        19496 non-null  object 
 5   tssloadrate_total     19496 non-null  object 
 6   tssloadate_conc       19496 non-null  object 
 7   catchment_hectares    19496 non-null  object 
 8   watershed_hectares    19496 non-null  object 
 9   tploadrate_total_ws   19496 non-null  object 
 10  tnloadrate_total_ws   19496 non-null  object 
 11  tssloadrate_total_ws  19496 non-null  object 
 12  maflowv               19496 non-null  object 
 13  geom                  19266 non-null  object 
 14  geom_catchment        19496 non-null  object 
 15  cluster            

In [29]:
base_df.head()

,comid,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,...,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
0,4149942,19.8106,0.0199,461.7835,0.4635,23315.2162,23.4000,199.4606,199.6200,0.0993,...,116.7978,1.1150,01050000E06A7F00000100000001020000C00E0000009D...,01060000206A7F000001000000010300000001000000AC...,Poconos and Kittatinny,None,NaN,72392.0,72392.0,020401040504
1,4151440,0.0037,0.0120,0.0679,0.2181,6.6330,12.7636,0.0899,5049.9000,0.0758,...,80.6635,35.7140,01050000E06A7F00000100000001020000C00300000003...,01060000206A7F00000100000001030000000100000005...,Poconos and Kittatinny,None,NaN,72128.0,72165.0,020401040504
2,4150082,2.0144,0.0099,25.4212,0.1405,4910.5669,22.9938,45.1439,69.2100,0.0490,...,113.7074,0.3830,01050000E06A7F00000100000001020000C00B0000000C...,01060000206A7F0000010000000103000000010000005F...,Poconos and Kittatinny,None,NaN,72440.0,72441.0,020401040504
3,4151394,2.9297,-9999.0000,58.2131,-9999.0000,5942.7803,-9999.0000,11.8705,697436.0100,-54432.3165,...,-54432.3165,4248.7040,01050000E06A7F00000100000001020000C00400000075...,01060000206A7F00000100000001030000000100000030...,Poconos and Kittatinny,None,NaN,72338.0,74971.0,020401040504
4,4149944,0.2212,0.0199,1.7217,0.4623,714.6338,23.9081,1.3489,200.9700,0.0993,...,119.2765,1.1220,01050000E06A7F00000100000001020000C0020000006F...,01060000206A7F0000010000000103000000010000000F...,Poconos and Kittatinny,None,NaN,72391.0,72392.0,020401040504


## Read from Database using SQLAlchemy and GeoPandas
An alternate way to connect, using sqlalchemy, written by Sarah. Benefits include:
- Doesn't require knowledge of column names
- Correctly auto-parses datatypes
- Auto-converts column named 'geom' from a string into a geometry datatype compatible with  GeoPandas and most plotting libraries.

In [40]:
%%time
# Connect to database with sqlalchemy
from sqlalchemy import create_engine  

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

CPU times: user 340 µs, sys: 5 µs, total: 345 µs
Wall time: 349 µs


In [41]:
%%time
# import data with geopandas compatible geometry
import geopandas as gpd 

tablename_base = 'base_run'
tablename_rest = 'restoration_run'

base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = gpd.read_postgis(base_model_select, con)
rest_df = gpd.read_postgis(rest_model_select, con)

CPU times: user 2.77 s, sys: 1.11 s, total: 3.89 s
Wall time: 37.9 s


In [42]:
base_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 19496 non-null  int64   
 1   tploadrate_total      19496 non-null  float64 
 2   tploadate_conc        19496 non-null  float64 
 3   tnloadrate_total      19496 non-null  float64 
 4   tnloadate_conc        19496 non-null  float64 
 5   tssloadrate_total     19496 non-null  float64 
 6   tssloadate_conc       19496 non-null  float64 
 7   catchment_hectares    19496 non-null  float64 
 8   watershed_hectares    19496 non-null  float64 
 9   tploadrate_total_ws   19496 non-null  float64 
 10  tnloadrate_total_ws   19496 non-null  float64 
 11  tssloadrate_total_ws  19496 non-null  float64 
 12  maflowv               19496 non-null  float64 
 13  geom                  19266 non-null  geometry
 14  geom_catchment        19496 non-null  object  

In [43]:
rest_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 19496 non-null  int64   
 1   tploadrate_total      19496 non-null  float64 
 2   tploadate_conc        19496 non-null  float64 
 3   tnloadrate_total      19496 non-null  float64 
 4   tnloadate_conc        19496 non-null  float64 
 5   tssloadrate_total     19496 non-null  float64 
 6   tssloadate_conc       19496 non-null  float64 
 7   catchment_hectares    19496 non-null  float64 
 8   watershed_hectares    19496 non-null  float64 
 9   tploadrate_total_ws   19496 non-null  float64 
 10  tnloadrate_total_ws   19496 non-null  float64 
 11  tssloadrate_total_ws  19496 non-null  float64 
 12  maflowv               19496 non-null  float64 
 13  geom                  19266 non-null  geometry
 14  geom_catchment        19496 non-null  object  

In [44]:
base_df.head()

,comid,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,...,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
0,4149942,19.8106,0.0199,461.7835,0.4635,23315.2162,23.4000,199.4606,199.62,0.0993,...,116.7978,1.115,MULTILINESTRING Z ((506490.373 4594278.694 0.0...,01060000206A7F000001000000010300000001000000AC...,Poconos and Kittatinny,None,NaN,72392.0,72392.0,020401040504
1,4151440,0.0037,0.0120,0.0679,0.2181,6.6330,12.7636,0.0899,5049.90,0.0758,...,80.6635,35.714,MULTILINESTRING Z ((514920.168 4587490.302 0.0...,01060000206A7F00000100000001030000000100000005...,Poconos and Kittatinny,None,NaN,72128.0,72165.0,020401040504
2,4150082,2.0144,0.0099,25.4212,0.1405,4910.5669,22.9938,45.1439,69.21,0.0490,...,113.7074,0.383,MULTILINESTRING Z ((502865.583 4590886.516 0.0...,01060000206A7F0000010000000103000000010000005F...,Poconos and Kittatinny,None,NaN,72440.0,72441.0,020401040504
3,4151394,2.9297,-9999.0000,58.2131,-9999.0000,5942.7803,-9999.0000,11.8705,697436.01,-54432.3165,...,-54432.3165,4248.704,MULTILINESTRING Z ((507356.841 4591522.967 0.0...,01060000206A7F00000100000001030000000100000030...,Poconos and Kittatinny,None,NaN,72338.0,74971.0,020401040504
4,4149944,0.2212,0.0199,1.7217,0.4623,714.6338,23.9081,1.3489,200.97,0.0993,...,119.2765,1.122,MULTILINESTRING Z ((506039.307 4593540.381 0.0...,01060000206A7F0000010000000103000000010000000F...,Poconos and Kittatinny,None,NaN,72391.0,72392.0,020401040504


In [45]:
# Set index to comid
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
base_df.set_index('comid', inplace=True)
rest_df.set_index('comid', inplace=True)

In [46]:
base_df.index

Int64Index([  4149942,   4151440,   4150082,   4151394,   4149944,   4150436,
              4150464,   4151392,   4150226,   4150518,
            ...
            932040262,   4150192,   2619260,   4188139,   4151472,   4150260,
              4151474,   4150526,   4151464,   4150466],
           dtype='int64', name='comid', length=19496)

In [47]:
rest_df.index

Int64Index([ 2612952,  2612782,  2612920,  2613460,  2612780,  2612792,
             2612956,  2612794,  2612948,  2612950,
            ...
             9437009, 26814149,  9437007,  9437021,  9436995,  9437027,
             9437011,  9436999,  8409259,  8409235],
           dtype='int64', name='comid', length=19496)

In [39]:
base_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 4149942 to 4150466
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   tploadrate_total      19496 non-null  float64 
 1   tploadate_conc        19496 non-null  float64 
 2   tnloadrate_total      19496 non-null  float64 
 3   tnloadate_conc        19496 non-null  float64 
 4   tssloadrate_total     19496 non-null  float64 
 5   tssloadate_conc       19496 non-null  float64 
 6   catchment_hectares    19496 non-null  float64 
 7   watershed_hectares    19496 non-null  float64 
 8   tploadrate_total_ws   19496 non-null  float64 
 9   tnloadrate_total_ws   19496 non-null  float64 
 10  tssloadrate_total_ws  19496 non-null  float64 
 11  maflowv               19496 non-null  float64 
 12  geom                  19266 non-null  geometry
 13  geom_catchment        19496 non-null  object  
 14  cluster               17358 non-null  

### Read again To Set Catchment Geometry
This reruns the `gpd.read_postgis()` function with the optional argument `geom_col="geom_catchment"`, as a workaround for converting both the flowlines and the catchment boundaries to geometry datatypes.

In [52]:
%%time
# get catchment geometry
base_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df_catch = gpd.read_postgis(base_catch, con, geom_col="geom_catchment")
rest_df_catch = gpd.read_postgis(rest_catch, con, geom_col="geom_catchment")

CPU times: user 3.92 s, sys: 693 ms, total: 4.61 s
Wall time: 25.5 s


In [53]:
base_df_catch.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   comid           19496 non-null  int64   
 1   geom_catchment  19496 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 304.8 KB


In [54]:
# Set index to comid
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
base_df_catch.set_index('comid', inplace=True)
rest_df_catch.set_index('comid', inplace=True)

### Replace `geom_catchment` Object with Geometry DType

In [55]:
# merge
base_df['geom_catchment'] = base_df_catch['geom_catchment']
rest_df['geom_catchment'] = rest_df_catch['geom_catchment']

In [56]:
base_df

,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,tnloadrate_total_ws,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,,,,,
4149942,19.8106,0.0199,461.7835,0.4635,23315.2162,23.4000,199.4606,199.62,0.0993,2.3135,116.7978,1.115,MULTILINESTRING Z ((506490.373 4594278.694 0.0...,"MULTIPOLYGON (((506092.122 4593422.061, 506098...",Poconos and Kittatinny,None,NaN,72392.0,72392.0,020401040504
4151440,0.0037,0.0120,0.0679,0.2181,6.6330,12.7636,0.0899,5049.90,0.0758,1.3784,80.6635,35.714,MULTILINESTRING Z ((514920.168 4587490.302 0.0...,"MULTIPOLYGON (((514955.572 4587467.785, 514926...",Poconos and Kittatinny,None,NaN,72128.0,72165.0,020401040504
4150082,2.0144,0.0099,25.4212,0.1405,4910.5669,22.9938,45.1439,69.21,0.0490,0.6948,113.7074,0.383,MULTILINESTRING Z ((502865.583 4590886.516 0.0...,"MULTIPOLYGON (((502581.690 4590660.525, 502522...",Poconos and Kittatinny,None,NaN,72440.0,72441.0,020401040504
4151394,2.9297,-9999.0000,58.2131,-9999.0000,5942.7803,-9999.0000,11.8705,697436.01,-54432.3165,-54432.3165,-54432.3165,4248.704,MULTILINESTRING Z ((507356.841 4591522.967 0.0...,"MULTIPOLYGON (((507205.238 4591376.554, 507218...",Poconos and Kittatinny,None,NaN,72338.0,74971.0,020401040504
4149944,0.2212,0.0199,1.7217,0.4623,714.6338,23.9081,1.3489,200.97,0.0993,2.3064,119.2765,1.122,MULTILINESTRING Z ((506039.307 4593540.381 0.0...,"MULTIPOLYGON (((506092.122 4593422.061, 505974...",Poconos and Kittatinny,None,NaN,72391.0,72392.0,020401040504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150260,1.2717,0.0147,17.4586,0.3105,3213.5741,24.8598,10.1619,488.88,0.0878,1.8542,148.4542,3.267,MULTILINESTRING Z ((511901.572 4588245.373 0.0...,"MULTIPOLYGON (((512065.727 4588106.375, 512006...",Poconos and Kittatinny,None,NaN,72178.0,72180.0,020401040504
4151474,0.0276,0.0154,0.6812,0.2186,42.7999,33.5127,0.6295,2902.50,0.1018,1.4445,221.4500,21.463,MULTILINESTRING Z ((509331.221 4587357.849 0.0...,"MULTIPOLYGON (((509438.231 4587314.755, 509379...",Poconos and Kittatinny,None,NaN,72182.0,72195.0,020401040504
4150526,18.9184,0.0145,479.5846,0.3683,23384.0050,17.9602,213.8491,214.02,0.0882,2.2406,109.2608,1.457,MULTILINESTRING Z ((506372.886 4582303.869 0.0...,"MULTIPOLYGON (((505993.004 4581580.731, 505934...",Poconos and Kittatinny,None,NaN,72191.0,72191.0,020401040504


In [63]:
base_df.attrs

{}

## Save to Parquet File
The code below converts the data into a locally saved parquet file to avoid having to access the database every time we run the visualization script.  

Apache Parquet has become the high-performance binary cloud format of choice for storing dataframes.
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html
- https://anaconda.org/TomAugspurger/pandas-performance/notebook

In [57]:
# Find current working directory
from pathlib import Path
Path.cwd()

PosixPath('/Users/aaufdenkampe/Documents/Python/WikiSRATMicroService')

In [58]:
# username = "Sarah"
username = "Anthony"

In [60]:
# Set alternate project & data folders
if username == "Anthony":
    project_folder = Path('/Users/aaufdenkampe/Documents/Python/WikiSRATMicroService')
elif username == "Sarah":
    project_folder = Path('C:/Users/sjordan/Documents/Python/WikiSRATMicroService')
data_folder    = Path('data/')

In [64]:
%%time

# requires pyarrow
import pyarrow
base_df.to_parquet(project_folder / data_folder /'base_df.parquet')
rest_df.to_parquet(project_folder / data_folder /'rest_df.parquet')

<timed exec>:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
<timed exec>:4: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this 

CPU times: user 3.36 s, sys: 54.9 ms, total: 3.41 s
Wall time: 3.43 s
